# Projeto 1 - Ciência dos Dados

Nome:Livia Brigido do Nascimento

Nome:Fabio Dotoli Ferreira Filho

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [39]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [40]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/fabiodff/Desktop/P1Cdados2021_2


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [41]:
filename = 'Airpods.xlsx'

In [50]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Treinamento,Classificacao_Treino
0,alguem me da um apple watch 😭😭😭😭 um macbook pr...,NaN
1,@hschmaiske @matheus1lva airpods pro é bom? é....,NaN
2,mais uns airpods estragados… que sorte 🤦🏻‍♂️,NaN
3,"@annaeso @masbahnana isso eu sei, mas nem espe...",NaN
4,sonhando com o dia que vou achar airpods por m...,NaN


In [54]:
train.Classificacao_Treino.value_counts(True)

Series([], Name: Classificacao_Treino, dtype: float64)

In [52]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificacao_Teste
0,"favela venceu, o pai lançou um airpods monstro...",0
1,"[weverse]\n\n“namjoon oppa, se eles lançarem a...",0
2,@priboechat comprei um fone tipo airpods e amei,0
3,@_army_brasileira_7 \n _army_brasileira_7 \n\n...,0
4,"@abelardo_moura ainda bem que foi o de fio, po...",0


In [53]:
test.Classificacao_Teste.value_counts(True)

0    0.713333
1    0.286667
Name: Classificacao_Teste, dtype: float64

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

### Produto escolhido ###
Os Airpods, lançamento da Apple no ano de 2016, são pequenos fones com conexão via bluetooth, que vêm acompanhados do estojo (usado para recarregá-los) e de um cabo lightning (carregador).

- Os tweets considerados relevantes foram aqueles que citavam alguma caracteristica relacionada ao produto escolhido, como por exemplo a durabilidade, a facilidade em perder os fones, preço e críticas/elogios aos Airpods como um todo.

- Os tweets irrelevantes, por sua vez, são aqueles que citam o produto, porém de uma forma que não é possivel extrair um feedback significativo sobre satisfação ou qualidades (positivas ou negativas) como, por exemplo, uma frase citando alguns itens aleatórios (dentre eles, os Airpods).

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [71]:
#Montando a função de limpeza principal e para a limpeza de palavras mais específicas** (cleanup)

import re #utilizaremos para limpar o texto ('replace')


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text) #substitui o primeiro argumento pelo segundo, e o terceiro argumento é onde será aplicada a substituição
    text_subbed = re.sub('\n', '', text_subbed) #retirando o enter (pular para a próxima linha)
    return text_subbed.lower

def cleanup_palavras(string): #**
    #vimos que haviam muitos links e marcações (@user), e que os mesmos poderiam atrapalhar no desempenho do classificador
    #Usando a ferramenta starwith de string:
    for i in string:
        if (i.startwith('https') == True) or (i.startwith('@') == True):
            i= re.sub(i, "", string) 
    return string


    

In [72]:
#Célula de Testes das funções:
variavel_teste= "@joniel,  gosta! do: https.hgahgsajd ?!"
variavel_limpa= cleanup_palavras (variavel_teste)
variavel_limpa

AttributeError: 'str' object has no attribute 'startwith'

#### Para os tweets Relevantes: ####

In [60]:
#perguntar como retirar os @ e oa nomes de usuários que os acompanham
#perguntar sobre os espaços entre palavras e emojis
#pesquisar sobre a suavização de laplace

In [62]:
#já foi dividido em rel. e irrel. pois depois não será possivel dividir por ser um df de apenas uma coluna 
train_rel_clean = train.loc[train["Classificacao_Treino"] == 1, 'Treinamento'].apply(cleanup) 
train_rel_clean.to_frame()

,Treinamento


In [63]:
#train_rel_clean.astype(str)
train_rel_lower = train_rel_clean.str.lower() #por que usar o .str ao inves de uma variavel-- Não quero trabalhar no dataset,mas em cada linha.
train_rel = train_rel_lower.str.split()

In [59]:
type(train_rel_clean)

NameError: name 'train_rel_clean' is not defined

In [74]:
train_rel_l = []  #criando uma lista para englobar todas as outras listas criadas com o .split

for palavras in train_rel:
    train_rel_l += palavras
serie_train_rel = pd.Series(train_rel_l)
serie_train_rel

0       @taecgiri
1             que
2         airpods
3           podre
4               é
          ...    
1412         caso
1413        perdi
1414          meu
1415      airpods
1416            🤡
Length: 1417, dtype: object

In [76]:
serie_train_rel_abs = serie_train_rel.value_counts(True)
serie_train_rel_abs

airpods        0.068454
que            0.025406
um             0.023994
o              0.023289
de             0.023289
                 ...   
abençoada      0.000706
desse          0.000706
gg             0.000706
pais           0.000706
sumirammmmm    0.000706
Length: 574, dtype: float64

#### Para os tweets Irrelevantes: ####

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**